In [19]:
# Mengimpor library
import os
import datetime
import numpy as np
import pandas as pd
from datetime import datetime

In [20]:
# Memuat dataset
list_of_dataframes = list()

# variabel untuk menyimpan data yang ada di folder input
players_df = None
competitions_df = None
games_df = None
game_events_df = None
club_games_df = None
player_valuations_df = None
game_lineups_df = None
appearances_df = None
clubs_df = None

# pengulangan melalui file-file di folder 'input' dan memuatnya ke dalam DataFrames
for dirname, _, filenames in os.walk('input'):
    for filename in filenames:
        # Buat nama DataFrame berdasarkan nama file
        df_name = filename.split('.')[0] + '_df'
        file = os.path.join(dirname, filename)
        
        # Baca file CSV dan buat DataFrame dengan nama dinamis
        df = pd.read_csv(file, sep=",", encoding="UTF-8")
        exec(f'{df_name} = df.copy()')
        print(file, df_name, df.shape)
        list_of_dataframes.append(df)


input\appearances.csv appearances_df (1518593, 13)
input\clubs.csv clubs_df (426, 17)
input\club_games.csv club_games_df (131306, 11)
input\competitions.csv competitions_df (43, 10)
input\games.csv games_df (65653, 23)
input\game_events.csv game_events_df (673431, 10)


C:\Users\Lenovo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


input\game_lineups.csv game_lineups_df (137258, 9)
input\players.csv players_df (30332, 23)
input\player_valuations.csv player_valuations_df (460537, 7)
input\test.csv test_df (341, 15)
input\train_data.csv train_data_df (1703, 7)


In [21]:

# Pastikan DataFrame players_df sudah diinisialisasi
# players_df = ...

# Pilih pemain dengan tanggal lahir yang kosong dan musim terakhir sama dengan 2023
selected_players = players_df[(players_df['date_of_birth'].isna()) & (players_df['last_season'] == 2023)]

# Tampilkan hasil seleksi
print(selected_players)


Empty DataFrame
Columns: [player_id, first_name, last_name, name, last_season, current_club_id, player_code, country_of_birth, city_of_birth, country_of_citizenship, date_of_birth, sub_position, position, foot, height_in_cm, contract_expiration_date, agent_name, image_url, url, current_club_domestic_competition_id, current_club_name, market_value_in_eur, highest_market_value_in_eur]
Index: []

[0 rows x 23 columns]


In [22]:
# Hitung umur setiap pemain dan sisa hari dalam kontrak mereka
# Hapus pemain yang tidak memiliki tanggal lahir atau tanggal kedaluwarsa kontrak
# Bulatkan umur ke bilangan bulat terdekat
players_df['date_of_birth'] = pd.to_datetime(players_df['date_of_birth'])
players_df = players_df[players_df['date_of_birth'].isnull() == False]
now = datetime.now()
players_df['age'] = (now - players_df['date_of_birth']).apply(lambda x: x.days) / 365.25
players_df['age'] = players_df['age'].round().astype(int)
players_df['contract_expiration_date'] = pd.to_datetime(players_df['contract_expiration_date'])
players_df = players_df[players_df['contract_expiration_date'].isnull() == False]
players_df['term_days_remaining'] = (players_df['contract_expiration_date'] - now).apply(lambda x: x.days)

In [23]:
# Buat DataFrame merged_players_df dengan menghapus beberapa kolom
# Reindeks kolom dan tambahkan kolom baru dari clubs_df
# Urutkan urutan kolom
merged_players_df = players_df.drop(['current_club_id', 'city_of_birth', 'date_of_birth', 'first_name', 'last_name', 'player_code', 'image_url', 'url'], axis=1)
merged_players_df = merged_players_df.reindex(columns=merged_players_df.columns.tolist() + ['club_value', 'squad_size', 'goals', 'goals_2022', 'games_2022', 'assists_2022', 'minutes_played_2022', 'goals_against_2022', 'goals_for_2022', 'clean_sheet_2022'])

# pengulangan melalui ID pemain untuk mengisi informasi terkait klub
for player_id in merged_players_df.player_id.unique():
    #print(players_df.current_club_id[(players_df.player_id==player_id)])
    club_id= players_df.current_club_id[(players_df.player_id==player_id)]
    #print(clubs_df.total_market_value[(clubs_df.club_id==int(club_id))])
    #print(clubs_df.squad_size[(clubs_df.club_id==int(club_id))])
    try:
        merged_players_df.club_value[(players_df.player_id==player_id)]=int(clubs_df.total_market_value[(clubs_df.club_id==int(club_id))])
    except:
        merged_players_df.club_value[(players_df.player_id==player_id)]='NaN'  
    merged_players_df.squad_size[(players_df.player_id==player_id)]=int((clubs_df.squad_size[(clubs_df.club_id==int(club_id))]))

columns=['player_id','games_2022','minutes_played_2022','goals_2022','assists_2022','goals_against_2022','goals_for_2022','clean_sheet_2022','name','position','sub_position','last_season','foot','height_in_cm','age','country_of_citizenship','country_of_birth','current_club_name','club_value','squad_size','current_club_domestic_competition_id','agent_name','contract_expiration_date','term_days_remaining','market_value_in_eur','highest_market_value_in_eur']
merged_players_df=merged_players_df[columns]

# Gabungkan DataFrames games dan appearances
games_and_appearances_df = appearances_df.merge(games_df, on=['game_id'], how='left')
print('Data Pemain dan Klub digabungkan')

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_2264/4198060440.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_players_df.club_value[(players_df.player_id==player_id)]='NaN'
C:\Users\Lenovo\AppData\Local\Temp/ipykernel_2264/4198060440.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_players_df.squad_size[(players_df.player_id==player_id)]=int((clubs_df.squad_size[(clubs_df.club_id==int(club_id))]))


Data Pemain dan Klub digabungkan


In [24]:
# Gabungkan kembali games dan appearances
games_and_appearances_df = appearances_df.merge(games_df, on=['game_id'], how='left')
season = 2022

# Buat fungsi untuk mengumpulkan statistik pemain berdasarkan player_id dan season
def player_stats(player_id, season, df):
    df = games_and_appearances_df[games_and_appearances_df['player_id'] == player_id]
    df =  df[ df['season'] == season]    
    if (df.shape[0] == 0):
        Out = [(np.nan, season,0,0,0,0,0,0,0,0,0)]
        out_df = pd.DataFrame(data = Out, columns = ['player_id','season','goals','games',
                                                     'assists','minutes_played','goals_for','goals_against','clean_sheet','yellow_cards','red_cards'])
        return out_df    
    else:       
        df["goals_for"] = df.apply(lambda row: row['home_club_goals'] if row['home_club_id'] == row['player_club_id'] 
            else row['away_club_goals'] if row['away_club_id'] == row['player_club_id'] 
            else np.nan, axis=1)
        df["goals_against"] = df.apply(lambda row: row['away_club_goals'] if row['home_club_id'] == row['player_club_id'] 
            else row['home_club_goals'] if row['away_club_id'] == row['player_club_id'] 
            else np.nan, axis=1)
        df['clean_sheet'] = df.apply(lambda row: 1 if row['goals_against'] == 0
            else 0 if row['goals_against'] > 0
            else np.nan, axis=1)
        df = df.groupby(['player_id',"season"],as_index=False).agg({'goals': 'sum', 'game_id': 'nunique', 
                                                                      'assists': 'sum', 'minutes_played' : 'sum', 'goals_for' : 'sum',
                                                                      'goals_against' : 'sum', 'clean_sheet' : 'sum','yellow_cards':'sum','red_cards':'sum'})
        out_df = df.rename(columns={'game_id': 'games'})
        return out_df

for index in merged_players_df.index:
    id = merged_players_df.loc[index][0]
    name = merged_players_df.loc[index][1]
    stats = player_stats(id, season, games_and_appearances_df)
    try:
        merged_players_df.at[index,'games_{}'.format(season)]= stats['games'][0]
        merged_players_df.at[index,'goals_{}'.format(season)]= stats['goals'][0]
        merged_players_df.at[index,'assists_{}'.format(season)]= stats['assists'][0]
        merged_players_df.at[index,'minutes_played_{}'.format(season)]= stats['minutes_played'][0]
        merged_players_df.at[index,'goals_for_{}'.format(season)]= stats['goals_for'][0]
        merged_players_df.at[index,'goals_against_{}'.format(season)]= stats['goals_against'][0]
        merged_players_df.at[index,'clean_sheet_{}'.format(season)]= stats['clean_sheet'][0]
        merged_players_df.at[index,'yellow_cards_{}'.format(season)]= stats['yellow_cards'][0]
        merged_players_df.at[index,'red_cards_{}'.format(season)]= stats['red_cards'][0]
    except:
        n=n+1

print('appearance,goal and card data merged')
print(merged_players_df.info())

appearance,goal and card data merged
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18881 entries, 9 to 30331
Data columns (total 28 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   player_id                             18881 non-null  int64         
 1   games_2022                            18881 non-null  float64       
 2   minutes_played_2022                   18881 non-null  float64       
 3   goals_2022                            18881 non-null  float64       
 4   assists_2022                          18881 non-null  float64       
 5   goals_against_2022                    18881 non-null  float64       
 6   goals_for_2022                        18881 non-null  float64       
 7   clean_sheet_2022                      18881 non-null  float64       
 8   name                                  18881 non-null  object        
 9   position                           

In [25]:
# Gabungkan merged_players_df dengan competitions_df berdasarkan 'current_club_domestic_competition_id'
suffixes = ('_player', '_competition')
dataProcessed = pd.merge(merged_players_df, competitions_df, left_on='current_club_domestic_competition_id', right_on='competition_id', how='left', suffixes=suffixes)

# Tampilkan informasi tentang dataframe yang digabungkan dan simpan ke file CSV
dataProcessed.to_csv('output/dataProcessed.csv', index=False)
print(dataProcessed.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 18881 entries, 0 to 18880
Data columns (total 38 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   player_id                             18881 non-null  int64         
 1   games_2022                            18881 non-null  float64       
 2   minutes_played_2022                   18881 non-null  float64       
 3   goals_2022                            18881 non-null  float64       
 4   assists_2022                          18881 non-null  float64       
 5   goals_against_2022                    18881 non-null  float64       
 6   goals_for_2022                        18881 non-null  float64       
 7   clean_sheet_2022                      18881 non-null  float64       
 8   name_player                           18881 non-null  object        
 9   position                              18881 non-null  object        
 10